# Projeto ADEL (Análise Dados Estadia em Limeira) - Webscraping das acomodações
Neste notebook, coletarei dados de acomodações anunciadas de algumas fontes. 

In [1]:
import pandas as pd

In [2]:
ss_sites = pd.read_csv('../Data/Imobiliarias/imobiliarias.csv', squeeze=True)

## Teste estático com BeautifulSoup

In [ ]:
from urllib.request import urlopen
import ssl

# This restores the same behavior as before.
context = ssl._create_unverified_context()
response = urlopen("url", context=context)
html_source = response.read()
soup = BeautifulSoup(html_source, 'html.parser')

## Coletando acomodações com Selenium - Parte 1

In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
from lxml import etree
from urllib.request import urlopen
import ssl
import hashlib
import webbrowser
import time

In [4]:
def add_row_df(df, columns, row):
    row_sr = pd.Series(row, index=columns)
    row_df = pd.DataFrame([row_sr])
    df = pd.concat([row_df, df], ignore_index=True)
    return df

def hash_data(data):
    return hashlib.md5(data.encode()).hexdigest()

def goto_imovel_url(imovel_url):
    context = ssl._create_unverified_context()
    response = urlopen(imovel_url, context=context)
    html_source = response.read()
    return BeautifulSoup(html_source, 'html.parser')

def use_xpath(soup):
    return etree.HTML(str(soup))

def xpath_get_text(dom, label, imovel_url, xpath):
    label += '\nAcesse o imovel: {}'.format(imovel_url)
    texto = dom.xpath(xpath)[0].text
    
    # Se não encontrar nada, coletar manualmente
    if not texto.split():
        texto = int(input(label))
        
    return texto

In [ ]:
soup_imovel_url = goto_imovel_url('url')
dom = use_xpath(soup_imovel_url)

In [11]:
# Dataframes
df_columns = [
    'imob',
    'imovel_url',
    'codigo', 
    'nome', 
    'bairro', 
    'aluguel',
    'condominio',
    'area',
    'quartos',
    'banheiros',
    'vaga_garagem',
    'descricao'
]
df_acomodacoes = pd.DataFrame(data=None, columns=df_columns)
df_acomodacoes_hashcode = pd.DataFrame(data=None, columns=df_columns)

site_index = 0

# Hide chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')

driver = webdriver.Chrome(options=chrome_options)
driver.get(ss_sites[site_index])

# Acessar página de kitnets
time.sleep(3)
driver.find_element_by_xpath('/html/body/main/div[5]/section/div[1]/div/div/div/div/div[1]/div[3]/div/div').click()
driver.find_element_by_xpath('/html/body/main/div[5]/section/div[1]/div/div/div/div/div[1]/div[3]/div/ul/li[11]/a').click()
driver.find_element_by_xpath('/html/body/main/div[5]/section/div[1]/div/div/div/div/div[1]/div[7]/a/div').click()

# Coletar os dados das kitnets
time.sleep(3)
pag = 1

# Avançar nas páginas
try:
    while True:
        print()
        print('-'*50)
        print('Página {}'.format(pag))
        print('-'*50)
        
        # Coletar dados estaticamente com BeautifulSoup
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        
        # Coletar dados de cada anúncio
        for acomodacao in soup.find_all('div', {'class': 'list__card jetgrid__col--6 jetgrid__col--md-8 jetgrid__col--sm-12 jetgrid__col--xs-24'}):
            imovel_url = ss_sites[site_index] + acomodacao.find('a')['href']
            cod = acomodacao.find('p', {'class': 'list__reference'}).get_text()
            nome = acomodacao.find('p', {'class': 'list__type'}).get_text()
            bairro = acomodacao.find('p', {'class': 'list__address'}).get_text()
            aluguel = acomodacao.find('div', {'class': 'list__price'}).get_text().strip()
            print(bairro)
            print(aluguel)
            
            # Pegar os demais dados
            soup_imovel_url = goto_imovel_url(imovel_url)
            time.sleep(3.5)
            #dom = use_xpath(soup_imovel_url)
            
            # Adicionar manualmente
            webbrowser.open_new_tab(imovel_url)
            
            condominio = float(input('Digite o valor do condomínio: '))
            area = int(input('Digite a área: '))
            num_quartos = int(input('Digite o número de quartos: '))
            num_banheiros = int(input('Digite o número de banheiros: '))
            vaga_garagem = input('Digite se há vaga de garagem: ')
            descricao = soup_imovel_url.find('p', {'class': 'card__text'}).get_text()
            
            # Adicionar registro
            df_acomodacoes = add_row_df(df_acomodacoes, df_columns,
                                                   [ss_sites[site_index], 
                                                    imovel_url,
                                                    cod,
                                                    nome, 
                                                    bairro,
                                                    aluguel,
                                                    condominio,
                                                    area,
                                                    num_quartos,
                                                    num_banheiros,
                                                    vaga_garagem,
                                                    descricao])
            df_acomodacoes_hashcode = add_row_df(df_acomodacoes_hashcode, df_columns,
                                                            [hash_data(ss_sites[site_index]),
                                                             hash_data(imovel_url),
                                                             hash_data(cod), 
                                                             hash_data(nome), 
                                                             bairro,
                                                             aluguel,
                                                             condominio,
                                                             area,
                                                             num_quartos,
                                                             num_banheiros,
                                                             vaga_garagem,
                                                             descricao])
            print('*'*30)
        
        # Parar quando não encontrar próxima página
        if driver.find_element_by_xpath('/html/body/main/div[5]/section/div[3]/div[2]/div[9]/div/div/a[2]').get_attribute('href') == None:
            break
            
        # Avançar para a próxima página
        driver.find_element_by_xpath('/html/body/main/div[5]/section/div[3]/div[2]/div[9]/div/div/a[2]').click()
        pag += 1
        time.sleep(3)
        
except:
    print('Fim')


--------------------------------------------------
Página 1
--------------------------------------------------
Jardim Cidade Universitaria I - Limeira/SP
Locação:  R$ 1.000,00
Digite o valor do condomínio: 0
Digite a área: 0
Digite o número de quartos: 1
Digite o número de banheiros: 1
Digite se há vaga de garagem: Não
******************************
Jardim Cidade Universitaria I - Limeira/SP
Locação:  R$ 1.000,00
Digite o valor do condomínio: 0
Digite a área: 0
Digite o número de quartos: 1
Digite o número de banheiros: 1
Digite se há vaga de garagem: Não
******************************
Jardim Cidade Universitaria I - Limeira/SP
Locação:  R$ 950,00
Digite o valor do condomínio: 0
Digite a área: 0
Digite o número de quartos: 1
Digite o número de banheiros: 1
Digite se há vaga de garagem: 1
******************************
Chacara Antonieta - Limeira/SP
Locação:  R$ 850,00
Digite o valor do condomínio: 0
Digite a área: 30
Digite o número de quartos: 1
Digite o número de banheiros: 1
Digite

In [12]:
df_acomodacoes.to_csv('../Data/Imobiliarias/acomodacoes.csv', index=False)
df_acomodacoes_hashcode.to_csv('../Data/acomodacoes_hashcode.csv', index=False)

In [13]:
df_acomodacoes_hashcode

,imob,imovel_url,codigo,nome,bairro,aluguel,condominio,area,quartos,banheiros,vaga_garagem,descricao
0,53dd1202c5ef8ce3878ffbd4b3c79bd2,05366ee9a81fea6a796a12010f338fde,c5a84c8eb6b4f9c7389fd32afa1e5357,c802e68178774c3218e92187307f113a,Jardim Cidade Universitaria I - Limeira/SP,"Locação: R$ 780,00",0.0,20,1,1,Não,"KITNET COM CAMA, ARMÁRIOS, ESCRIVANINHA,COZINH..."
1,53dd1202c5ef8ce3878ffbd4b3c79bd2,351b093210805519c0e16eea800955c3,9b5801ddc55b2411554d6fb109229460,c802e68178774c3218e92187307f113a,Jardim Cidade Universitaria I - Limeira/SP,"Locação: R$ 800,00",0.0,20,1,1,0,"COZINHA COM GELADEIRA COM DISPENSER NA PORTA, ..."
2,53dd1202c5ef8ce3878ffbd4b3c79bd2,22434d2e45f882427cdb19c48b93f947,f773fe72ae9c3ae06183ea91d181347a,c802e68178774c3218e92187307f113a,Jardim Cidade Universitaria I - Limeira/SP,"Locação: R$ 800,00",80.0,20,1,1,0,"COZINHA COM GELADEIRA COM DISPENSER NA PORTA, ..."
3,53dd1202c5ef8ce3878ffbd4b3c79bd2,440fb0d94ac1b708a18764608c269abd,350e21033122ce335fd0fd5f0031179d,c802e68178774c3218e92187307f113a,Jardim Cidade Universitaria I - Limeira/SP,"Locação: R$ 800,00",0.0,20,1,1,Sim,"COZINHA COM GELADEIRA COM DISPENSER NA PORTA, ..."
4,53dd1202c5ef8ce3878ffbd4b3c79bd2,a1c5e82b12d23e634843ef2f1dc597eb,13a10985cb92f1112ce6894b29a575e7,c802e68178774c3218e92187307f113a,Chacara Antonieta - Limeira/SP,"Locação: R$ 850,00",0.0,30,1,1,1,"CAMA, GUARDA-ROUPA, GELADEIRA, BANCADA, TANQU..."
5,53dd1202c5ef8ce3878ffbd4b3c79bd2,4ddc1cae387d65ac833577f059c6d628,a92a6e35acca298c33b9d2c83bde3505,c802e68178774c3218e92187307f113a,Chacara Antonieta - Limeira/SP,"Locação: R$ 850,00",0.0,30,1,1,Sim,"CAMA, GUARDA-ROUPA, GELADEIRA, BANCADA, TANQU..."
6,53dd1202c5ef8ce3878ffbd4b3c79bd2,4491337d5ace9995c23f9ac16d850de8,4b4adce0a87af0846fda6bdbe38d670a,c802e68178774c3218e92187307f113a,Jardim Cidade Universitaria I - Limeira/SP,"Locação: R$ 950,00",0.0,0,1,1,1,"KITNET CONTENDO FOGÃO, GELADEIRA, ARMÁRIO EMBU..."
7,53dd1202c5ef8ce3878ffbd4b3c79bd2,14f56a71eb61773b57dff76a0b56a462,57c6964ba64836c81a7a7a782a4d0eb1,c802e68178774c3218e92187307f113a,Jardim Cidade Universitaria I - Limeira/SP,"Locação: R$ 1.000,00",0.0,0,1,1,Não,1 QUARTO COM ARMÁRIO E AR CONDICIONADO;1 SALA ...
8,53dd1202c5ef8ce3878ffbd4b3c79bd2,0338ae8955c864ba896df28bc4e5f039,4cc0e019beccf19b1e1182b9e896f089,c802e68178774c3218e92187307f113a,Jardim Cidade Universitaria I - Limeira/SP,"Locação: R$ 1.000,00",0.0,0,1,1,Não,1 QUARTO COM ARMÁRIO E AR CONDICIONADO;1 SALA ...
